### Este notebook contem todo o processo de mineração do dataset de casas em King County, EUA

#### Carregamento dos dados

In [1]:
from pysrc.loading import *

In [2]:
dataframe.sample(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
13423,622100074,20141110T000000,720000,3,2.75,2440,76531,1.0,0,0,...,10,1640,800,1980,0,98072,47.7672,-122.161,1890,10042
12680,925069123,20150318T000000,590000,3,1.00,1610,58370,1.0,0,0,...,7,1610,0,1978,0,98053,47.6718,-122.044,2510,58127
18393,3879901285,20150326T000000,1230000,3,2.50,2660,1967,3.0,0,3,...,9,1870,790,2007,0,98119,47.6264,-122.363,1640,1369
5281,1126059007,20150323T000000,865000,3,2.25,2670,150270,2.0,0,0,...,9,2670,0,1985,0,98072,47.7601,-122.134,3080,81054
12577,2806300065,20150422T000000,1960000,4,4.00,4430,31353,2.0,0,0,...,12,4430,0,1998,0,98005,47.6422,-122.157,3900,35237


In [3]:
# Colunas da base de dados, inicialmente
showInitialCols()

21 colunas
['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


## Preprocessamento

In [4]:
from pysrc.preprocessing import *

#### Remover colunas que não nos interessam

As colunas removidas são impressas abaixo

In [5]:
dropUselessCols()

In [6]:
# Colunas restantes
showFinalCols()

16 colunas restantes
Removidas  ['id', 'date', 'zipcode', 'lat', 'long']

 ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']


#### Verificar qualidade dos dados restantes

In [7]:
# Checando premissas fundamentais sobre os atributos
# Valores nao mencionados abaixo foram checados individualmente
# Nao pensei der necessario inclui-los abaixo
checkDataRanges()

Zero valores faltantes
Todos os valores sao nao-negativos
Anos de contrucao entre 1900 e 2015
Ano de renovacao >= ano de contrucao
Ano de renovacao <= 2015


#### Aplicar algumas tranformações convenientes

In [8]:
# Substituir casos em que yr_renovated é igual a zero pelo ano
#   de construção da casa.
affectedInstances = dataframe[dataframe["yr_renovated"] == 0].index
dataframe.loc[affectedInstances, "yr_renovated"] = dataframe.loc[affectedInstances, "yr_built"]

In [14]:
# Normalizar dataframe por tecnica Min-Max
normalize

In [9]:
# Categorizar tamanhos
